## Radio interferometría y síntesis de imágenes en astronomía - Laboratorio 2

### Vicente Mieres

In [ ]:
# Modulos
import sys
import os

route = os.path.abspath(os.path.join("..", "modules"))
sys.path.append(route)